in here we will be building an alarm system/motion detection based off of these three videos. the plan is to make an alarm system that captures the frame where the motion was detected mostly all the frames containing motion detection

we are first going to import 
cv2 for webcam image handling
imutils for cropping resizing
threading to handle multiple frames
winsound for alarm system

In [1]:
import cv2
import imutils
import threading
import winsound

we then setup the webcamfeed variable, height and width

In [2]:
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


True

we will get a frame and an immediate netxt frame. if there is enough differences between the frame we will sound the allarm
so, we define a start frame, set its width to 500, convert the frame wo grayscale to view changes, then we use a gausian blur on the image to check for deviation

In [5]:
ret, start_frame = cap.read()
start_frame = imutils.resize(start_frame, width=500)
start_frame = cv2.cvtColor(start_frame, cv2.COLOR_BGR2GRAY)
start_frame = cv2.GaussianBlur(start_frame, (21, 21), 0)

variable that check if an alarm is active, alarm mode  to check if alarm is toggled and alarm countr to set duration of alarm

In [6]:
alarm = False
alarm_mode = False
alarm_counter = 0


define a simple alarm function that defines what happens when an alarm is set off
windows.beep sets the frequency pitch of the sound
we want to produce 5 consecutive one second beeps

In [7]:
def beep_alarm():
    global alarm
    for _ in range(5):
        if not alarm_mode:
            break
        print("Alarm")
        winsound.Beep(2500, 1000)
    alarm = False




In [ ]:
while True:
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=500)

    if alarm_mode:
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (5, 5), 0)

        difference = cv2.absdiff(frame_bw, start_frame)
        threshold = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]

        if threshold.sum() > 300:
            alarm_counter += 1
        else:
            if alarm_counter > 0:
                alarm_counter -= 1

        cv2.imshow("Cam", threshold)
    else:
        cv2.imshow("Cam", frame)

    if alarm_counter > 20 :
        if not alarm:
            alarm = True
            threading.Thread(target=beep_alarm).start()

    key_pressed = cv2.waitKey(30)
    if key_pressed == ord("t"):
        alarm_mode = not alarm_mode
        alarm_counter = 0
    if key_pressed == ord("q"):
        alarm_mode = False
        break

cap.release()
cv2.destroyAllWindows()
        